In [2]:
import requests


class ApeksAPI(object):
    URL = 'https://avtorvuz.orurinst.site'
    TOKEN = 'b41dfa22-0a72-477f-9f05-995b8409863a'
    APEKS_DEPT_ID = '4'  # ID of Department divisions type

def db_request(dbname):
    """DB request function without filter"""
    params = {"token": ApeksAPI.TOKEN, "table": dbname}
    response = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params
    )
    return response.json()["data"]


def db_filter_req(dbname, sqltable, sqlvalue):
    """Filtered DB request (DB table, filter, value)"""
    params = {
        "token": ApeksAPI.TOKEN,
        "table": dbname,
        "filter[" + sqltable + "]": str(sqlvalue),
    }
    response = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params
    )
    return response.json()["data"]


def plan_curriculum_disciplines(education_plan_id):
    """Getting disciplines info as dict (disc_ID:[disc_code:disc_name])"""
    disciplines = {}
    resp = db_filter_req(
        "plan_curriculum_disciplines", "education_plan_id", education_plan_id
    )
    for disc in resp:
        if disc["level"] == "3":
            disciplines[disc["id"]] = [
                disc["code"],
                db_filter_req("plan_disciplines", "id", disc["discipline_id"])[0]["name"],
            ]
    return disciplines

def education_plans(education_specialty_id, year='0'):
    """Getting education plans with selected speciality"""
    payload = {
        "token": ApeksAPI.TOKEN,
        "table": "plan_education_plans",
        "filter[data_type]": "plan",
        "filter[education_specialty_id]": education_specialty_id,
        "filter[active]": "1",
    }
    request = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=payload
    )
    plans = {}
    if year == '0':
        for i in request.json()["data"]:
            plans[i.get("id")] = i.get("name")
        return plans
    else:
        for i in request.json()["data"]:
            if i.get("custom_start_year") == str(year):
                plans[i.get("id")] = i.get("name")
            elif i.get("custom_start_year") is None:
                if db_filter_req('plan_semesters', 'education_plan_id', i.get("id"))[0]['start_date'].split('-')[0] == str(year):
                    plans[i.get("id")] = i.get("name")
        return plans

In [6]:
from app.programs.models import WorkProgram

wp = WorkProgram(10465)

In [7]:
wp.get('tasks')

IndexError: list index out of range